In [31]:
# import dataset
def main():
    data_file_name = input('Data file name: ')
    data_type_file_name = input('Column type file name: ')
    df = pd.read_csv('../../data/processed/'+data_file_name+'.csv')
    df_type = pd.read_csv('../../data/processed/'+data_type_file_name+'.csv')
    Linear_Regression(dataset, dataset_type)

In [28]:
#build linear regression model
def Linear_Regression(dataset,dataset_type,test_size=0.2):
    import pandas as pd
    import numpy as np
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split
    import statsmodels.formula.api as stfa
    from sklearn.metrics import mean_squared_error
    
    print('The current dataset has',dataset.shape[1],'columns and',dataset.shape[0],'rows')
    print('Column names are',dataset.columns)
    print('')
    print('The first 10 rows of this dataset:')
    print(dataset.head(10))
    
    # funtion to get column type
    def column_type(column_name,df_type):
        return (df_type.loc[df_type['Variable'] == column_name, 'Type'].iloc[0])
    #Function to get target variable
    def get_target(df,df_type):
        for c in df:
            if (column_type(c,df_type) == 'Flag_Continuous' or column_type(c,df_type) == 'Flag_Categorical'):
                return(c)
    y=get_target(dataset,dataset_type)
    # seperate train set (80%) and test set (20%)
    d_train, d_test = train_test_split(dataset, test_size=test_size, random_state=0)
    dx_train = d_train.drop([y],axis =1)
    dy_train = d_train[[y]]
    dx_test = d_test.drop([y],axis =1)
    dy_test = d_test[[y]]
    # create a list to store simple LP model train and test error
    column_list = dx_train.columns.values.tolist()
    # get formula
    formula_list = y + '~'
    for column in column_list:
        formula_list = formula_list + ' + ' + column
    # fit simple LP model
    SLP = stfa.ols(formula = formula_list, data = d_train).fit()
    ytest_pred1 = SLP.predict(dx_test)
    test_error = mean_squared_error(dy_test, ytest_pred1, sample_weight=None, multioutput='uniform_average')
    SLP_sum = SLP.summary()
    print(SLP_sum)
    print('')
    print(test_error)

In [29]:
Linear_Regression(dataset,dataset_type)

The current dataset has 15 columns and 5000 rows
Column names are Index(['Calendar_Year', 'Model_Year', 'Blind_Make', 'Cat1', 'Cat2', 'Cat3',
       'Cat6', 'Cat8', 'Cat9', 'Cat10', 'Cat11', 'Cat12', 'OrdCat', 'NVCat',
       'Claim_Amount'],
      dtype='object')

The first 10 rows of this dataset:
   Calendar_Year  Model_Year  Blind_Make  Cat1  Cat2  Cat3  Cat6  Cat8  Cat9  \
0              1           2           3     1     2     2     2     0     0   
1              1           2           0     1     2     5     1     2     1   
2              1           2           1     1     2     5     2     2     1   
3              0           2           1     1     2     5     2     2     1   
4              1           1           0     0     2     4     4     2     1   
5              0           1           0     0     2     4     4     2     1   
6              0           1           2     0     2     5     1     2     1   
7              0           2           0     1     2     4 

In [30]:
# save the model to disk
import pickle
filename = '../../models/LinearRegressor.sav'
pickle.dump(stfa.ols, open(filename, 'wb'))